In [6]:
%run import_data.ipynb

Dataset books.csv loaded successfully
isbn column dropped, isbn13 column kept
Language codes eng, en-US, en-GB, en-CA unified to 'en'
Missing publication dates filled in
2nd (large) dataset is being downloaded...
Dataset downloaded and saved as data/books2.csv
Dataset downloaded and saved as data/users.csv
Dataset downloaded and saved as data/ratings.csv
Pandas dataframes (books_df, books_big, users, ratings) loaded successfully
Columns renamed and dates converted to dtype: datetime
Ready to go!


In [7]:
# import requests
# import pandas as pd

# def get_book_annotation(isbn13):
#     # Define the API endpoint
#     url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn13}'
    
#     # Make the API request
#     response = requests.get(url)
    
#     # Check if the request was successful
#     if response.status_code == 200:
#         data = response.json()
        
#         # Check if any books were found
#         if 'items' in data:
#             # Extract the description from the first item
#             annotation = data['items'][0]['volumeInfo'].get('description', 'No annotation found')
#             return annotation
#         else:
#             return 'No annotation found'
#     else:
#         return 'Error fetching annotation'

# Example usage:
# print(get_book_annotation('9780140449136'))  # Replace with an actual ISBN13


In [32]:
smol_df = books_df.sample(100)
smol_df.head()

,bookid,title,authors,average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
10164,41098,World of Wonders (The Deptford Trilogy #3),Robertson Davies/Wayne Johnston,4.09,9780143039143,en,352,3633,142,2006-02-28,Penguin Classics
3786,13611,ヒカルの碁 12、新初段シリーズ,Yumi Hotta/Yumi Hotta,4.24,9784088731100,jpn,189,9,2,2001-05-06,集英社
3251,11918,Bitten (Otherworld #1),Kelley Armstrong,4.05,9780452286030,en,436,79710,3479,2004-09-07,Plume Books
2749,10134,Wie ich eines schönen Morgens im April das 100...,Haruki Murakami/Nora Bierich,3.80,9783499222504,ger,218,44,2,1998-08-01,Rowohlt Tb
7785,30039,The Prize (deWarenne Dynasty #4),Brenda Joyce,3.98,9780778320890,en,575,1604,66,2004-09-24,Mira Books


In [9]:
# fetch annotations for each book and create a new column
# handle API rate limits by adding a delay between requests
# import time

# def get_book_annotation(isbn13):
#     url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn13}'
#     response = requests.get(url)
    
#     # Respect the rate limit
#     time.sleep(1)
    
#     if response.status_code == 200:
#         data = response.json()
#         if 'items' in data:
#             annotation = data['items'][0]['volumeInfo'].get('description', 'No annotation found')
#             return annotation
#         else:
#             return 'No annotation found'
#     else:
#         return 'Error fetching annotation'

# smol_df['annotation'] = smol_df['isbn13'].apply(get_book_annotation)

In [33]:
smol_df.head(10)

,bookid,title,authors,average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
10164,41098,World of Wonders (The Deptford Trilogy #3),Robertson Davies/Wayne Johnston,4.09,9780143039143,en,352,3633,142,2006-02-28,Penguin Classics
3786,13611,ヒカルの碁 12、新初段シリーズ,Yumi Hotta/Yumi Hotta,4.24,9784088731100,jpn,189,9,2,2001-05-06,集英社
3251,11918,Bitten (Otherworld #1),Kelley Armstrong,4.05,9780452286030,en,436,79710,3479,2004-09-07,Plume Books
2749,10134,Wie ich eines schönen Morgens im April das 100...,Haruki Murakami/Nora Bierich,3.80,9783499222504,ger,218,44,2,1998-08-01,Rowohlt Tb
7785,30039,The Prize (deWarenne Dynasty #4),Brenda Joyce,3.98,9780778320890,en,575,1604,66,2004-09-24,Mira Books
8977,34873,The Great And Secret Show (Book of the Art #1),Clive Barker,4.05,9780006176954,en,698,275,17,1990-04-26,Fontana Books
7330,28228,Double Fold: Libraries and the Assault on Paper,Nicholson Baker,3.42,9780375726217,en,370,846,91,2002-04-09,Vintage
7691,29603,20 000 Leagues Under the Sea,Ron Miller/Jules Verne/Paul Wright,3.76,9780751370737,en,64,34,4,1998-10-08,Prentice Hall
7922,30371,Temperatures Rising,Sandra Brown,3.45,9780553804089,en,214,2059,80,2006-11-28,Bantam Books
6577,24781,The High King (The Chronicles of Prydain #5),Lloyd Alexander,4.25,9780805080520,en,253,48088,954,2006-05-16,Square Fish


In [11]:
# smol_df['annotation'].iloc[9]

'Now in paperback! Here is the book that updates the rulebook, giving parents the training and skills they need to transform their teenage children into strong, confident, productive adults.'

In [45]:
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio
from asyncio import Semaphore
import logging

# Apply the nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Enable logging for debugging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

# Set the maximum number of concurrent requests (adjust based on the API rate limit)
MAX_CONCURRENT_REQUESTS = 5
semaphore = Semaphore(MAX_CONCURRENT_REQUESTS)

async def fetch_annotation(session, isbn13):
    async with semaphore:
        url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn13}'
        try:
            async with session.get(url) as response:
                logger.info(f'Fetching ISBN: {isbn13} with status: {response.status}')
                if response.status == 429:  # Too many requests
                    logger.warning(f'Rate limit hit for ISBN: {isbn13}, retrying after 60 seconds')
                    await asyncio.sleep(60)  # Wait 60 seconds and try again
                    return await fetch_annotation(session, isbn13)  # Retry
                elif response.status == 200:
                    data = await response.json()
                    logger.debug(f'Received data for ISBN: {isbn13}: {data}')
                    # key = f'ISBN:{isbn13}'
                    if 'items' in data:
                        annotation = data['items'][0]['volumeInfo'].get('description', 'No annotation found')
                        return annotation if annotation else 'No annotation found'
                    else:
                        return 'No annotation found'
                else:
                    logger.error(f'Error fetching ISBN: {isbn13}, status code: {response.status}')
                    return 'Error fetching annotation'
        except Exception as e:
            logger.error(f'Exception occurred for ISBN: {isbn13}: {e}')
            return 'Error fetching annotation'

async def fetch_annotations_for_dataframe(smol_df, rate_limit=5):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for isbn13 in smol_df['isbn13']:
            tasks.append(fetch_annotation(session, isbn13))
            if len(tasks) % rate_limit == 0:  # After every 'rate_limit' requests
                await asyncio.sleep(1)  # Sleep to respect the rate limit
        annotations = await asyncio.gather(*tasks)
    smol_df['annotation'] = annotations
    return smol_df

async def main():
    global smol_df
    # Assuming smol_df is your existing DataFrame with an 'isbn13' column
    smol_df = await fetch_annotations_for_dataframe(smol_df)
    print(smol_df.head())  # Display the updated DataFrame with annotations

# Instead of asyncio.run(main()), we use an event loop directly:
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


INFO:root:Fetching ISBN: 9780452286030 with status: 200
INFO:root:Fetching ISBN: 9784088731100 with status: 200
INFO:root:Fetching ISBN: 9780778320890 with status: 200
INFO:root:Fetching ISBN: 9780143039143 with status: 200
INFO:root:Fetching ISBN: 9783499222504 with status: 200
INFO:root:Fetching ISBN: 9780006176954 with status: 200
INFO:root:Fetching ISBN: 9780375726217 with status: 200
INFO:root:Fetching ISBN: 9780805080520 with status: 200
INFO:root:Fetching ISBN: 9780751370737 with status: 200
INFO:root:Fetching ISBN: 9780553804089 with status: 200
INFO:root:Fetching ISBN: 9780613371650 with status: 200
INFO:root:Fetching ISBN: 9780439573887 with status: 200
INFO:root:Fetching ISBN: 9780932813862 with status: 200
INFO:root:Fetching ISBN: 9783551552716 with status: 200
INFO:root:Fetching ISBN: 9780140158762 with status: 200
INFO:root:Fetching ISBN: 9781585675746 with status: 200
INFO:root:Fetching ISBN: 9780060938109 with status: 200
INFO:root:Fetching ISBN: 9780345492791 with stat

       bookid                                              title  \
10164   41098        World of Wonders (The Deptford Trilogy  #3)   
3786    13611                                   ヒカルの碁 12、新初段シリーズ   
3251    11918                            Bitten (Otherworld  #1)   
2749    10134  Wie ich eines schönen Morgens im April das 100...   
7785    30039                  The Prize (deWarenne Dynasty  #4)   

                               authors  average_rating         isbn13  \
10164  Robertson Davies/Wayne Johnston            4.09  9780143039143   
3786             Yumi Hotta/Yumi Hotta            4.24  9784088731100   
3251                  Kelley Armstrong            4.05  9780452286030   
2749      Haruki Murakami/Nora Bierich            3.80  9783499222504   
7785                      Brenda Joyce            3.98  9780778320890   

      language_code  num_pages  ratings_count  text_reviews_count  \
10164            en        352           3633                 142   
3786          

In [46]:
smol_df.head(10)

,bookid,title,authors,average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,annotation
10164,41098,World of Wonders (The Deptford Trilogy #3),Robertson Davies/Wayne Johnston,4.09,9780143039143,en,352,3633,142,2006-02-28,Penguin Classics,The third book in Robertson Davies's acclaimed...
3786,13611,ヒカルの碁 12、新初段シリーズ,Yumi Hotta/Yumi Hotta,4.24,9784088731100,jpn,189,9,2,2001-05-06,集英社,No annotation found
3251,11918,Bitten (Otherworld #1),Kelley Armstrong,4.05,9780452286030,en,436,79710,3479,2004-09-07,Plume Books,"On the eve of her marriage, Elena Michaels lea..."
2749,10134,Wie ich eines schönen Morgens im April das 100...,Haruki Murakami/Nora Bierich,3.80,9783499222504,ger,218,44,2,1998-08-01,Rowohlt Tb,No annotation found
7785,30039,The Prize (deWarenne Dynasty #4),Brenda Joyce,3.98,9780778320890,en,575,1604,66,2004-09-24,Mira Books,AN INFAMOUS SEA CAPTAIN OF THE BRITISH ROYAL N...
8977,34873,The Great And Secret Show (Book of the Art #1),Clive Barker,4.05,9780006176954,en,698,275,17,1990-04-26,Fontana Books,"In the little town of Palomo Grove, two great ..."
7330,28228,Double Fold: Libraries and the Assault on Paper,Nicholson Baker,3.42,9780375726217,en,370,846,91,2002-04-09,Vintage,The ostensible purpose of a library is to pres...
7691,29603,20 000 Leagues Under the Sea,Ron Miller/Jules Verne/Paul Wright,3.76,9780751370737,en,64,34,4,1998-10-08,Prentice Hall,This is a two-colour dictionary and CD-ROM for...
7922,30371,Temperatures Rising,Sandra Brown,3.45,9780553804089,en,214,2059,80,2006-11-28,Bantam Books,"Darkly handsome with an arrogant edge, archite..."
6577,24781,The High King (The Chronicles of Prydain #5),Lloyd Alexander,4.25,9780805080520,en,253,48088,954,2006-05-16,Square Fish,Taran and his allies carry the battle to the g...


In [47]:
smol_df['annotation'].nunique() # 88 out of our 100 book sample have annotations

89

In [ ]:
# books_df.to_csv('books_with_annotations.csv', index=False)